In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon


In [4]:
#Task 1: Mark all launch sites on a map

In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [9]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
launch_sites = [
    {"name": "CCAFS LC-40", "lat": 28.562302, "long": -80.577356},
    {"name": "CCAFS SLC-40", "lat": 28.563197, "long": -80.576820},
    {"name": "KSC LC-39A", "lat": 28.573255, "long": -80.646895},
    {"name": "VAFB SLC-4E", "lat": 34.632834, "long": -120.610745}
]

# Loop through each launch site and add Circle and Marker
for site in launch_sites:
    # Create a Circle with a popup showing the launch site name
    circle = folium.Circle(
        location=[site['lat'], site['long']], 
        radius=1000,  # Radius in meters
        color='#2980b9',  # Circle color (blue)
        fill=True,
        fill_color='#2980b9',  # Fill color (blue)
        fill_opacity=0.5  # Transparency of the circle
    ).add_child(folium.Popup(site['name']))  # Add a popup to the circle

    # Create a Marker with a text label using DivIcon
    marker = folium.map.Marker(
        location=[site['lat'], site['long']], 
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#2980b9;"><b>%s</b></div>' % site['name'],
        )
    )
    
    # Add the circle and marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)


# Display the map
site_map

In [10]:
# Task 2: Mark the success/failed launches for each site on the map

In [11]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [12]:
marker_cluster = MarkerCluster()

In [13]:
# Step 2: Create the 'marker_color' column based on the 'class' value
# Assign marker colors based on the class column (1 for success, 0 for failure)
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

In [14]:
# Step 3: Iterate through spacex_df and create markers
for index, record in spacex_df.iterrows():
    # Create a Marker object for each row in spacex_df
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],  # Latitude and Longitude from the DataFrame
        popup=f"Launch Site: {record['Launch Site']}<br>Class: {record['class']}",  # Popup content
        icon=folium.Icon(color='white', icon_color=record['marker_color'])  # Use the marker_color for icon color
    )
    
    # Step 5: Add this marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Step 6: Add the MarkerCluster to the map
site_map.add_child(marker_cluster)

site_map

In [15]:
# TASK 3: Calculate the distances between a launch site to its proximities

In [16]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [17]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [18]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
distance_coastline = calculate_distance(28.563197, -80.576820, 28.56367, -80.56794)

In [19]:
 # Create a Marker with a text label using DivIcon
distance_marker = folium.map.Marker(
    location= [28.56367, -80.56794] , 
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)
# Add the marker to the map
distance_marker.add_to(site_map)

#display map
site_map

In [20]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# Create a list of coordinates for the polyline (launch site + closest coastline)
# list of Coordinates
launch_site_lat = 28.563197
launch_site_lon = -80.576820
closest_coast_lat = 28.56367
closest_coast_lon = -80.56794

coordinates = [
    [launch_site_lat, launch_site_lon],
    [closest_coast_lat, closest_coast_lon]
]

lines=folium.PolyLine(locations=coordinates, weight=1)

site_map.add_child(lines)

site_map

In [22]:
# find railway distance
distance_railway = calculate_distance(28.563197, -80.576820, 28.56374, -80.58682)

# Create a Marker with a text label using DivIcon
distance_marker_r = folium.map.Marker(
    location= [28.56374, -80.58682] , 
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
    )
)
# Add the marker to the map
distance_marker_r.add_to(site_map)

#Create a `folium.PolyLine` object using the railway and launch site coordinate
# list of Coordinates
launch_site_lat = 28.563197
launch_site_lon = -80.576820
railway_lat = 28.56374
railway_lon = -80.58682

coordinates_1 = [
    [launch_site_lat, launch_site_lon],
    [railway_lat, railway_lon]
]

lines=folium.PolyLine(locations=coordinates_1, weight=1)

site_map.add_child(lines)

site_map

In [23]:
# find highway distance
distance_highway = calculate_distance(28.563197, -80.576820, 28.56341, -80.57084)

# Create a Marker with a text label using DivIcon
distance_marker_h = folium.map.Marker(
    location= [28.56341, -80.57084] , 
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
    )
)
# Add the marker to the map
distance_marker_h.add_to(site_map)

#Create a `folium.PolyLine` object using the railway and launch site coordinate
# list of Coordinates
launch_site_lat = 28.563197
launch_site_lon = -80.576820
highway_lat = 28.56341
highway_lon = -80.57084

coordinates_2 = [
    [launch_site_lat, launch_site_lon],
    [highway_lat, highway_lon]
]

lines=folium.PolyLine(locations=coordinates_2, weight=1)

site_map.add_child(lines)

site_map

In [24]:
# find closest city distance
distance_city = calculate_distance(28.563197, -80.576820, 28.38717, -80.6012)

# Create a Marker with a text label using DivIcon
distance_marker_c = folium.map.Marker(
    location= [28.38717, -80.6012] , 
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
    )
)
# Add the marker to the map
distance_marker_c.add_to(site_map)

#Create a `folium.PolyLine` object using the railway and launch site coordinate
# list of Coordinates
launch_site_lat = 28.563197
launch_site_lon = -80.576820
city_lat = 28.38717
city_lon = -80.6012

coordinates_3 = [
    [launch_site_lat, launch_site_lon],
    [city_lat, city_lon]
]

lines=folium.PolyLine(locations=coordinates_3, weight=1)

site_map.add_child(lines)

site_map